In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, HiveContext

# Read & Write avec HDFS

In [2]:
sc = SparkContext()

In [3]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)

In [4]:
# Read from hdfs
textFile = sc.textFile("hdfs://192.168.56.10:8020/user/hdfs/test/results1.csv")
textFile.first()

'(GAVIN,9)'

In [ ]:
# Write with hdfs
distData.saveAsTextFile("hdfs://192.168.56.10:8020/user/hdfs/test/distData.csv")

# Read & Write avec Hive

In [5]:
SparkContext.setSystemProperty("hive.metastore.uris", "thrift://nn1:9083")
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [6]:
hive_context = HiveContext(spark)
hive_context.sql("show tables").show()

+--------+----------------+-----------+
|database|       tableName|isTemporary|
+--------+----------------+-----------+
| default|        contrats|      false|
| default|contrats_non_res|      false|
| default|    contrats_res|      false|
| default|          df_mvt|      false|
| default|         df_mvt1|      false|
| default|     earthquakes|      false|
| default|            mrsq|      false|
| default|     mvtprod2015|      false|
| default|            test|      false|
+--------+----------------+-----------+



In [10]:
hive_context.sql('from contrats select *')

DataFrame[getdatedemanderes: timestamp, mrsq_anc_ref: string, mrsq_cont: string, mrsq_data_dat: timestamp, mrsq_dres_cnt: timestamp, mrsq_cres_cnt: string]

# Autres tests

In [11]:
# spark is an existing SparkSession
df = spark.read.json("hdfs://192.168.56.10:8020/user/hdfs/test/customers_example.json")
# Displays the content of the DataFrame to stdout
df.show()

+--------------------+----------+----------+
|             address|first_name| last_name|
+--------------------+----------+----------+
|[New Orleans,LA,6...|     James|Butterburg|
|[Brighton,MI,4 B ...| Josephine|   Darakjy|
|[Bridgeport,NJ,8 ...|       Art|    Chemel|
+--------------------+----------+----------+



In [18]:
# A tester : lien xml
df = spark.read.format("hdfs://192.168.56.10:8020/user/hdfs/test/books.xml")

In [20]:
df.show()

AttributeError: 'DataFrameReader' object has no attribute 'show'